In [1]:
from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
import os
from tempfile import NamedTemporaryFile, TemporaryDirectory # Creating temporary Files/Dirs
from subprocess import run, PIPE
import sys
 
import dask # Distributed data libary
from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
from distributed import Client, progress, wait # Libaray to orchestrate distributed resources

import xarray as xr # Libary to work with labeled n-dimensional data and dask
import numpy as np
import matplotlib.pyplot as plt
# import metpy.calc as mpcalc

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
dask_client = Client()
dask_client

Client Scheduler: tcp://127.0.0.1:35380 Dashboard: http://127.0.0.1:40285/status,Cluster Workers: 2 Cores: 2 Memory: 2.68 GB


In [7]:
data_path = Path('/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/')

glob_pattern_2d = 'div900_*reggrid.nc' 
div_files = sorted([str(f) for f in data_path.rglob(f'{glob_pattern_2d}')])[-31:]
glob_pattern_2d = '*mask.nc'
mask_files = sorted([str(f) for f in data_path.rglob(f'{glob_pattern_2d}')])

In [8]:
div_files

['/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200131T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200201T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200202T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200203T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200204T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200205T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200206T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200207T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200208T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200209T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_20200210T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div900_2020021

In [9]:
var = xr.open_mfdataset(div_files)
# var = xr.open_mfdataset('/work/mh1119/DYAMOND_winter/sst-sic-runmean_0017_R02B10.nc', chunks={'time': 10})['SST'].squeeze()
# mask = xr.open_mfdataset(mask_files)['high_rome_mask'].squeeze()

In [10]:
var

<xarray.Dataset>
Dimensions:  (lat: 1777, lon: 16000, time: 211)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-31T03:00:00 ... 2020-03-01
  * lon      (lon) float64 -180.0 -180.0 -179.9 -179.9 ... 179.9 180.0 180.0
  * lat      (lat) float64 -19.99 -19.97 -19.94 -19.92 ... 19.93 19.95 19.97
    plev     float64 9e+04
Data variables:
    div      (time, lat, lon) float64 dask.array<chunksize=(7, 1777, 16000), meta=np.ndarray>

In [7]:
var.time

<xarray.DataArray 'time' (time: 200)>
array(['2020-01-13T00:00:00.000000000', '2020-01-13T06:00:00.000000000',
       '2020-01-13T12:00:00.000000000', '2020-01-13T18:00:00.000000000',
       '2020-01-14T00:00:00.000000000', '2020-01-14T06:00:00.000000000',
       '2020-01-14T12:00:00.000000000', '2020-01-14T18:00:00.000000000',
       '2020-01-15T00:00:00.000000000', '2020-01-15T06:00:00.000000000',
       '2020-01-15T12:00:00.000000000', '2020-01-15T18:00:00.000000000',
       '2020-01-16T00:00:00.000000000', '2020-01-16T06:00:00.000000000',
       '2020-01-16T12:00:00.000000000', '2020-01-16T18:00:00.000000000',
       '2020-01-17T00:00:00.000000000', '2020-01-17T06:00:00.000000000',
       '2020-01-17T12:00:00.000000000', '2020-01-17T18:00:00.000000000',
       '2020-01-18T00:00:00.000000000', '2020-01-18T06:00:00.000000000',
       '2020-01-18T12:00:00.000000000', '2020-01-18T18:00:00.000000000',
       '2020-01-19T00:00:00.000000000', '2020-01-19T06:00:00.000000000',
       '2020-01-19T12:00:00.000000000', '2020-01-19T18:00:00.000000000',
       '2020-01-20T00:00:00.000000000', '2020-01-20T06:00:00.000000000',
       '2020-01-20T12:00:00.000000000', '2020-01-20T18:00:00.000000000',
       '2020-01-21T00:00:00.000000000', '2020-01-21T06:00:00.000000000',
       '2020-01-21T12:00:00.000000000', '2020-01-21T18:00:00.000000000',
       '2020-01-22T00:00:00.000000000', '2020-01-22T06:00:00.000000000',
       '2020-01-22T12:00:00.000000000', '2020-01-22T18:00:00.000000000',
       '2020-01-23T00:00:00.000000000', '2020-01-23T06:00:00.000000000',
       '2020-01-23T12:00:00.000000000', '2020-01-23T18:00:00.000000000',
       '2020-01-24T00:00:00.000000000', '2020-01-24T06:00:00.000000000',
       '2020-01-24T12:00:00.000000000', '2020-01-24T18:00:00.000000000',
       '2020-01-25T00:00:00.000000000', '2020-01-25T06:00:00.000000000',
       '2020-01-25T12:00:00.000000000', '2020-01-25T18:00:00.000000000',
       '2020-01-26T00:00:00.000000000', '2020-01-26T06:00:00.000000000',
       '2020-01-26T12:00:00.000000000', '2020-01-26T18:00:00.000000000',
       '2020-01-27T00:00:00.000000000', '2020-01-27T06:00:00.000000000',
       '2020-01-27T12:00:00.000000000', '2020-01-27T18:00:00.000000000',
       '2020-01-28T00:00:00.000000000', '2020-01-28T06:00:00.000000000',
       '2020-01-28T12:00:00.000000000', '2020-01-28T18:00:00.000000000',
       '2020-01-29T00:00:00.000000000', '2020-01-29T06:00:00.000000000',
       '2020-01-29T12:00:00.000000000', '2020-01-29T18:00:00.000000000',
       '2020-01-30T00:00:00.000000000', '2020-01-30T06:00:00.000000000',
       '2020-01-30T12:00:00.000000000', '2020-01-30T18:00:00.000000000',
       '2020-01-31T00:00:00.000000000', '2020-01-31T06:00:00.000000000',
       '2020-01-31T12:00:00.000000000', '2020-01-31T18:00:00.000000000',
       '2020-02-01T00:00:00.000000000', '2020-02-01T06:00:00.000000000',
       '2020-02-01T12:00:00.000000000', '2020-02-01T18:00:00.000000000',
       '2020-02-02T00:00:00.000000000', '2020-02-02T06:00:00.000000000',
       '2020-02-02T12:00:00.000000000', '2020-02-02T18:00:00.000000000',
       '2020-02-03T00:00:00.000000000', '2020-02-03T06:00:00.000000000',
       '2020-02-03T12:00:00.000000000', '2020-02-03T18:00:00.000000000',
       '2020-02-04T00:00:00.000000000', '2020-02-04T06:00:00.000000000',
       '2020-02-04T12:00:00.000000000', '2020-02-04T18:00:00.000000000',
       '2020-02-05T00:00:00.000000000', '2020-02-05T06:00:00.000000000',
       '2020-02-05T12:00:00.000000000', '2020-02-05T18:00:00.000000000',
       '2020-02-06T00:00:00.000000000', '2020-02-06T06:00:00.000000000',
       '2020-02-06T12:00:00.000000000', '2020-02-06T18:00:00.000000000',
       '2020-02-07T00:00:00.000000000', '2020-02-07T06:00:00.000000000',
       '2020-02-07T12:00:00.000000000', '2020-02-07T18:00:00.000000000',
       '2020-02-08T00:00:00.000000000', '2020-02-08T06:00:00.000000000',
       '2020-02-08T12:00:00.000000000', '2020-02-08T18:00:00.000000000',
       '2020-02-09T0

In [8]:
# mask

In [9]:
var_slice = var.sel(time=slice('2020-01-31T00', '2020-03-01T00'))
var_mean = var_slice.mean(dim='time')
# div_mean_romep90 = div.where(mask, other=np.nan).mean(dim='time')
# div_mean_romep90 = div.where(mask, other=np.nan).sum(dim='time') / len(div['time'])
var_mean

<xarray.DataArray 'SST' (cell: 83886080)>
dask.array<mean_agg-aggregate, shape=(83886080,), dtype=float32, chunksize=(83886080,), chunktype=numpy.ndarray>
Coordinates:
    clon     (cell) float64 dask.array<chunksize=(83886080,), meta=np.ndarray>
    clat     (cell) float64 dask.array<chunksize=(83886080,), meta=np.ndarray>
Dimensions without coordinates: cell

In [10]:
# div_day = div[:-1].groupby('time.time').mean()
# div_day['time'] = div[:7]['time']
# import pandas as pd
# tt = pd.to_datetime(div.time.values)
# tt.hour == 0
# div_day.to_netcdf('/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div_dailycycle.nc')

In [11]:
var_mean.to_netcdf('/work/mh0731/m300414/DyWinter_b10/sst_timemean.nc')